In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

### Task 1

Cargue el conjunto de datos de Iris utilizando bibliotecas como sklearn.datasets. Luego, divida el conjunto de datos  en conjuntos de entrenamiento y validación. 

In [4]:
iris = load_iris()
X = iris.data         
y = iris.target    

# Normalizar características
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print("Tamaño del set de entrenamiento:", len(train_dataset))
print("Tamaño del set de validación:", len(val_dataset))


Tamaño del set de entrenamiento: 105
Tamaño del set de validación: 45


### Task 2

Cree una red neuronal feedforward simple utilizando nn.Module de PyTorch. Luego, defina capa de entrada, capas  ocultas y capa de salida. Después, elija las funciones de activación y el número de neuronas por capa. 

In [7]:
class FeedforwardNN(nn.Module):
    def __init__(self, input_dim=4, hidden1=16, hidden2=8, output_dim=3):
        super(FeedforwardNN, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))   
        x = self.relu(self.fc2(x))  
        x = self.fc3(x)            
        return x

model = FeedforwardNN()

print(model)


FeedforwardNN(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=3, bias=True)
  (relu): ReLU()
)


### Task 3

Utilice diferentes funciones de pérdida comunes como Cross-Entropy Loss y MSE para clasificación. Entrene el  modelo con diferentes funciones de pérdida y registre las pérdidas de entrenamiento y test. Debe utilizar al menos 3  diferentes funciones. Es decir, procure que su código sea capaz de parametrizar el uso de diferentes funciones de  pérdida.  

In [9]:
def train_model(loss_fn_name, train_loader, val_loader, epochs=50, lr=0.01):
    input_dim, output_dim = 4, 3
    model = FeedforwardNN(input_dim=input_dim, output_dim=output_dim)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    loss_functions = {
        "cross_entropy": nn.CrossEntropyLoss(),
        "mse": nn.MSELoss(),
        "nll": nn.NLLLoss()
    }
    criterion = loss_functions[loss_fn_name]

    history = {"train_loss": [], "val_loss": []}

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()

            outputs = model(X_batch)

            if loss_fn_name == "mse":
                y_batch_oh = nn.functional.one_hot(y_batch, num_classes=output_dim).float()
                loss = criterion(outputs, y_batch_oh)
            elif loss_fn_name == "nll":
                log_probs = nn.functional.log_softmax(outputs, dim=1)
                loss = criterion(log_probs, y_batch)
            else:
                loss = criterion(outputs, y_batch)

            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_val, y_val in val_loader:
                outputs = model(X_val)

                if loss_fn_name == "mse":
                    y_val_oh = nn.functional.one_hot(y_val, num_classes=output_dim).float()
                    loss = criterion(outputs, y_val_oh)
                elif loss_fn_name == "nll":
                    log_probs = nn.functional.log_softmax(outputs, dim=1)
                    loss = criterion(log_probs, y_val)
                else:
                    loss = criterion(outputs, y_val)

                val_loss += loss.item()

        history["train_loss"].append(train_loss / len(train_loader))
        history["val_loss"].append(val_loss / len(val_loader))

        if (epoch + 1) % 10 == 0:
            print(f"[{loss_fn_name}] Epoch {epoch+1}/{epochs} "
                  f"Train Loss: {history['train_loss'][-1]:.4f}, "
                  f"Val Loss: {history['val_loss'][-1]:.4f}")

    return history

losses = {}
for loss_name in ["cross_entropy", "mse", "nll"]:
    print(f"\nEntrenando con {loss_name.upper()}...\n")
    history = train_model(loss_name, train_loader, val_loader, epochs=50, lr=0.01)
    losses[loss_name] = history


Entrenando con CROSS_ENTROPY...

[cross_entropy] Epoch 10/50 Train Loss: 0.2961, Val Loss: 0.3506
[cross_entropy] Epoch 20/50 Train Loss: 0.0591, Val Loss: 0.1258
[cross_entropy] Epoch 30/50 Train Loss: 0.0302, Val Loss: 0.1497
[cross_entropy] Epoch 40/50 Train Loss: 0.0266, Val Loss: 0.2220
[cross_entropy] Epoch 50/50 Train Loss: 0.0306, Val Loss: 0.1532

Entrenando con MSE...

[mse] Epoch 10/50 Train Loss: 0.0229, Val Loss: 0.0309
[mse] Epoch 20/50 Train Loss: 0.0176, Val Loss: 0.0256
[mse] Epoch 30/50 Train Loss: 0.0145, Val Loss: 0.0241
[mse] Epoch 40/50 Train Loss: 0.0122, Val Loss: 0.0213
[mse] Epoch 50/50 Train Loss: 0.0116, Val Loss: 0.0204

Entrenando con NLL...

[nll] Epoch 10/50 Train Loss: 0.1611, Val Loss: 0.2519
[nll] Epoch 20/50 Train Loss: 0.0397, Val Loss: 0.1157
[nll] Epoch 30/50 Train Loss: 0.0242, Val Loss: 0.1115
[nll] Epoch 40/50 Train Loss: 0.0134, Val Loss: 0.1128
[nll] Epoch 50/50 Train Loss: 0.0147, Val Loss: 0.1470
